# Working With Models

## Register Existing Model

The following example registers a model from a local file.

In [ ]:
from azureml.core import Workspace
from azureml.core import Model
import sklearn

ws = Workspace.from_config()
model = Model.register(workspace = ws,
    model_name='mnb-cooking-life',
    tags={'kind': 'demo'},
    model_path='./output/mnb_model.pkl',
    model_framework = Model.Framework.SCIKITLEARN,
    model_framework_version = sklearn.__version__)

# This is how you could delete the model
# model.delete()

# Working With Deployments

The following code sample deploys the previously registered model to
[*Azure Container Instances*](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-azure-container-instance). Other
deployment targets are available. Choose based on the size of your project
([guidance](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where)).

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 2)
service = Model.deploy(ws, 'my-deployed-model', [model], deployment_config = deployment_config)

Here you see how we can remove a deployment:

In [ ]:
from azureml.core import Workspace
from azureml.core.webservice import Webservice

ws = Workspace.from_config()
webserv = Webservice(ws, 'mnb-cooking-live-deployment')
webserv.delete()

# Training Models

You can [train models](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-set-up-training-targets) in various ways in AzureML.
We start with a simple version by training our model on the same compute instance that we use for Jupyter.

Note that AzureML makes heavy use of container technology. Real-world training is done in containers. The base container image
[is available on Docker Hub](https://github.com/Azure/AzureML-Containers). You can build your own images if you need to.

In [ ]:
from azureml.core import Experiment

ws = Workspace.from_config()
experiment_name = 'IngredientsToCultureRainer'
experiment = Experiment(workspace=ws, name=experiment_name)

In [ ]:
from azureml.core import Environment
from azureml.core import ScriptRunConfig
import os

myenv = Environment("user-managed-env")
myenv.python.user_managed_dependencies = True

src = ScriptRunConfig(source_directory = os.getcwd(),
    script = 'train.py',
    environment = myenv)

run = experiment.submit(config=src)
run.wait_for_completion(show_output=True)

## Training on Compute Clusters

For the next step, you have to [create a compute cluster](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python).
Next, we [create an environment](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-set-up-training-targets#create-an-environment).

In [ ]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment('cooking-test-env')
cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults', 'unidecode', 'nltk'], conda_packages=['scikit-learn==0.22.1'])

env.python.conda_dependencies = cd

# Register environment to re-use later
ws = Workspace.from_config()
env.register(workspace = ws)

Once we got the environment, we can [setup the script run configuration](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-set-up-training-targets#create-the-script-run-configuration).

In [ ]:
from azureml.core import ScriptRunConfig
from azureml.core.compute import ComputeTarget
from azureml.core import Experiment
import os

ws = Workspace.from_config()
compute_target = ComputeTarget(ws, 'cpu-cluster')

src = ScriptRunConfig(source_directory = os.getcwd(),
    script = 'train.py', 
    compute_target = compute_target,
    environment = env)

experiment_name = 'cooking-on-cluster'
exp = Experiment(workspace = ws, name = experiment_name)


Now it is time to run the experiment:

In [ ]:
run = exp.submit(config = src)
run

We can monitor the log here in our Jupyter notebook:

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()